In [123]:

#    Multi-Disciplinary Design Optimisation of a Hydrogen Powered Commercial Aircraft (MDOHPCA)
#    this is the master file that runs each of the individual disciplines 
#    It sets up each of the analyses, then converts it into an openMDAO problem with a certain structure, then solves for minimum mass



In [136]:
import openmdao.api as om
import pandas as pd
import numpy as np

In [137]:
class MDOHPCA(om.Group):
    """
    Top level group containing the MDO.
    """
    def setup(self):
        """
        set up each person's disciplines that can contain analyses
        """
        self.add_subsystem('aero', aero())
        self.add_subsystem('struct', struct())
        self.add_subsystem('sys', sys())
        self.add_subsystem('stab', stab()) 
        


In [138]:
%run struct/structsetup.ipynb
%run stab/stabsetup.ipynb
%run systems/systemssetup.ipynb


In [139]:
prob = om.Problem(model = MDOHPCA())
#prob.model.add_subsystem('aero', om.Group())

In [140]:
prob.driver = om.ScipyOptimizeDriver()
prob.driver.options['optimizer'] = 'SLSQP'
# prob.driver.options['maxiter'] = 100
prob.driver.options['tol'] = 1e-8
prob.driver.options["debug_print"] = ["nl_cons", "objs", "desvars"]

In [141]:
prob.setup()

In [142]:
%run aero/aerosetup.ipynb

In [143]:
prob.run_model()

ValueError: 'aero.aero_point_0.aero_states.solve_matrix' <class SolveMatrix>: Error calling solve_nonlinear(), array must not contain infs or NaNs

In [8]:
#get properties of specific constants in disciplines
prob.model.sys.options._dict['pp_mass']

{'val': 32816.0,
 'values': None,
 'types': None,
 'desc': 'people and cargo mass',
 'upper': None,
 'lower': None,
 'check_valid': None,
 'has_been_set': True,
 'allow_none': False,
 'recordable': True,
 'deprecation': None}

In [145]:
prob.model.aero.list_outputs()

53 Explicit Output(s) in 'aero'

varname                       val                 
----------------------------  --------------------
prob_vars
  v                           [-238.07]           
  alpha                       [0.]                
  Mach_number                 [0.91243685]        
  re                          [2646937.24515845]  
  rho                         [0.12765534]        
  cg                          |0.0|               
wing
  twist_bsp
    twist                     |0.0|               
  chord_bsp
    chord                     |23.76088267|       
  t_over_c_bsp
    t_over_c                  |0.31758152|        
  mesh
    taper
      mesh                    |240.01175069|      
    scale_x
      mesh                    |240.98102562|      
    sweep
      mesh                    |251.31544695|      
    shear_x
      mesh                    |251.31544695|      
    stretch
      mesh                    |251.31544695|      
    shear_y
      mesh            

[('prob_vars.v', {'val': array([-238.07])}),
 ('prob_vars.alpha', {'val': array([0.])}),
 ('prob_vars.Mach_number', {'val': array([0.91243685])}),
 ('prob_vars.re', {'val': array([2646937.24515845])}),
 ('prob_vars.rho', {'val': array([0.12765534])}),
 ('prob_vars.cg', {'val': array([0., 0., 0.])}),
 ('wing.twist_bsp.twist', {'val': array([[0., 0., 0., 0., 0., 0., 0., 0.]])}),
 ('wing.chord_bsp.chord',
  {'val': array([[8.40074063, 8.40074063, 8.40074063, 8.40074063, 8.40074063,
           8.40074063, 8.40074063, 8.40074063]])}),
 ('wing.t_over_c_bsp.t_over_c',
  {'val': array([[0.12003453, 0.12003453, 0.12003453, 0.12003453, 0.12003453,
           0.12003453, 0.12003453]])}),
 ('wing.mesh.taper.mesh',
  {'val': array([[[-3.75000000e-01, -4.00000000e+01,  0.00000000e+00],
           [-3.78134011e-01, -3.89971165e+01,  0.00000000e+00],
           [-3.87378892e-01, -3.60387547e+01,  0.00000000e+00],
           [-4.02271065e-01, -3.12732593e+01,  0.00000000e+00],
           [-4.22063775e-